# Segunda Parte de Revisión de código para 2.1 One-sided Jacobi numerical aproximación

**Fecha:** 12 de Abril de 2020

**Responsable de revisión:** Dorely Morales

**Código revisado**

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices


}

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
    # u, v como vectores de la misma dimensión.Y un valor real de tolerancia TOL(10^-8).
    # Nota: Se sugiere una TOL mayor a 10^-32.
  # Returns: 
    # Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){
        ret<-0
    } else{ 
        
        if( (u%*%v)  /(norm(u,type ="2")*norm(v,type ="2")) < TOL   ){
    ret<-1
  }
  else{
    ret<-0
  }  
        
    }
  ret
}

In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, para construir x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #  	    U (mxm),V(nxn), S(mxn) matriz diagonal y b (m) un vector.
    # Returns: 
    #      x vector (m)
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

In [5]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Función que calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representan a las matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicialza valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0

    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
  
    ind <- indices(n)
    for(i in 1:nmax){
      col_j<-ak[,ind[[i]][[1]][2]]
      col_i<-ak[,ind[[i]][[1]][1]]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-sum(col_i*col_i)
        b<-sum(col_j*col_j)
        c<-col_i%*%col_j
        
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        for(l in seq(1,m)){
          temp<-ak[l,ind[[i]][[1]][1]]
          ak[l,ind[[i]][[1]][1]]<-cs*temp-sn*ak[l,ind[[i]][[1]][2]]
          ak[l,ind[[i]][[1]][2]]<-sn*temp+cs*ak[l,ind[[i]][[1]][2]]
        }
        
        #actualiza las columnas de la matriz vk
        for(l in seq(1,n)){
          temp<-vk[l,ind[[i]][[1]][1]]
          vk[l,ind[[i]][[1]][1]]<-cs*temp-sn*vk[l,ind[[i]][[1]][2]]
          vk[l,ind[[i]][[1]][2]]<-sn*temp+cs*vk[l,ind[[i]][[1]][2]]
        
      }
    }
  }
  k<-k+1
}
    #Obtener sigma
        for(i in 1:n){
        sig<- append(sig,norm(ak[,i],type ="2"))
    }

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            
            uk[,i]<-0
            
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }

**3. Pruebas**

Ocupe la presente sección para hacer diseño de pruebas variando los parámetros que recibe el código la función en diferentes rangos para evaluar su comportamiento y/o detectar posibles fallos


**1.Sobre la documentación del código/de la función**

¿Se encuentran presentes en la implementación los siguientes elementos? Por favor, ingrese explicaciones detalladas.

**a) Descripción concisa y breve de lo que hace el código/la función**

Sí.

**b) Descripción de sus argumentos de entrada, su significado y rango de valores que pueden tomar**

Sí.

**c) Descripción de los tipos de argumentos de entrada y de salida (por ejemplo, valores enteros, reales, strings, dataframe, matrices, etc)**

Sí.

**d) Descripción de la salida de la función, su significado y valores/objetos que deben regresar**

Sí.

**2. Cumplimiento de objetivos del código/de la función**

**a) ¿El código cumple los objetivos para los que fue diseñado?**

Sí.

**b) ¿La salida de la función genera el valor necesario?**

Parcialmente, es necesario corregir algunos errores que ocurren cuando se introduce una matriz renglón y matriz columna. 

**Test 6**

**Objetivo del test 6:** Verificar que la función se comporta correctamente para A **matriz renglón**.

**Implementación del test 6:**

In [79]:
#Definimos A matriz de 1x10
A <- matrix(c(1:10), nrow=1, ncol=10, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

ERROR: Error in svd(x, nu = 0L, nv = 0L): infinite or missing values in 'x'


**Principales hallazgos del test 6**

La función arroja el mismo error que el mencionado en test 1 y test 2 del archivo "Rev_One-sided_Jacobi.ipynb".

**Test 7**

**Objetivo del test 7:** Verificar que la función se comporta correctamente para A **matriz columna**.

**Implementación del test 7:**

In [6]:
#Definimos A matriz de 2x1, es decir n=2, m=1
A <- matrix(c(1:2), nrow=2, ncol=1, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

Aunque la función no genera un error, si intentamos comprobar que se obtuvo la SVD de A, obtenemos un error.

In [7]:
A
svd$U%*%svd$S%*%t(svd$V)

1
2


ERROR: Error in svd$U %*% svd$S %*% t(svd$V): non-conformable arguments


Al revisar la lista con las matrices S, U y V, encontramos que las dimensiones de S no coinciden con las salidas que debe generar la función. Es decir, si n=2 y m=1 S debe ser de 1x1, U de 1x2 y V de 1x1. Sin embargo, S regresa una matriz identidad de 2x2 aunque U y V sí son correctos.

In [8]:
svd

1,0
0,1


Si sustituimos por el valor correcto de S, obtenemos la misma matriz A.

In [10]:
Scorr <- matrix(c(2.2360679774998), nrow=1, ncol=1, byrow = T)
svd$U%*%Scorr%*%t(svd$V)

1
2


Ahora repetiremos el ejercicio para una matriz A de 3x1:

In [13]:
#Definimos A matriz de 3x1, es decir n=3, m=1
A <- matrix(c(1:3), nrow=3, ncol=1, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [14]:
A
svd$U%*%svd$S%*%t(svd$V)

1
2
3


ERROR: Error in svd$U %*% svd$S %*% t(svd$V): non-conformable arguments


Al revisar la lista con las matrices S, U y V, encontramos que las dimensiones de S nuevamente no coinciden con las salidas que debe generar la función. Es decir, si n=3 y m=1 S debe ser de 1x1, U de 1x3 y V de 1x1. Sin embargo, S regresa una matriz identidad de 3x3 aunque U y V sí son correctos.

In [15]:
svd

1,0,0
0,1,0
0,0,1


Si sustituimos por el valor correcto de S, obtenemos la misma matriz A.

In [16]:
Scorr <- matrix(c(3.7416573867739), nrow=1, ncol=1, byrow = T)
svd$U%*%Scorr%*%t(svd$V)

1
2
3


**Principales hallazos del test 7**

La función genera una lista con S, U y V para matrices renglón A de nx1. Sin embargo, las dimensiones de S corresponden a la matriz identidad de nxn en lugar del valor singular sigma.

**Test 8**

**Objetivo del test 8:** Verificar que la función se comporta correctamente para A **matriz simétrica**

**Implementación del test 8:**

In [21]:
#Definimos A matriz columna de 3x3
A <- matrix(c(5,1/3,pi,1/3,3,-7,pi,-7,8), nrow=3, ncol=3, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [22]:
A
svd$U%*%svd$S%*%t(svd$V)

5.0000000,0.3333333,3.141593
0.3333333,3.0000000,-7.000000
3.1415927,-7.0000000,8.000000


5.0000000,0.3333333,3.141593
0.3333333,3.0000000,-7.000000
3.1415927,-7.0000000,8.000000


**Principales hallazos del test 8**

La función arroja resultados correctos encontrando la SVD de A.

**Test 9**

**Objetivo del test 9:** Verificar que la función se comporta correctamente para A **matriz diagonal**

**Implementación del test 9:**

In [23]:
#Definimos A matriz diagonal
A <- matrix(c(10**30,0,0,0,0,-1,0,0,0,0,-1/3,0,0,0,0,log(2)), nrow=4, ncol=4, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [24]:
A
svd$U%*%svd$S%*%t(svd$V)

1e+30,0,0.0000000,0.0000000
0e+00,-1,0.0000000,0.0000000
0e+00,0,-0.3333333,0.0000000
0e+00,0,0.0000000,0.6931472


1e+30,0,0.0000000,0.0000000
0e+00,-1,0.0000000,0.0000000
0e+00,0,-0.3333333,0.0000000
0e+00,0,0.0000000,0.6931472


**Principales hallazos del test 9**

La función arroja resultados correctos encontrando la SVD de A.

**Test 10**

**Objetivo del test 10:** Verificar que la función se comporta correctamente para matrices A **triangular superior e inferior**.

**Implementación del test 10:**

In [37]:
#Definimos A matriz triangular superior
A <- matrix(c(3,1,2,0,4,3,0,0,6), nrow=3, ncol=3, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [42]:
A
svd$U%*%svd$S%*%t(svd$V)

3,1,2
0,4,3
0,0,6


3.000000e+00,1.000000e+00,2
-4.440892e-16,4.000000e+00,3
8.326673e-17,1.873501e-16,6


In [51]:
#Definimos A matriz triangular inferior
A <- matrix(c(1/7,0,0,0,1,4,0,0,pi,-2,sqrt(2),0,sqrt(2),3,0,0), nrow=4, ncol=4, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [52]:
A
svd$U%*%svd$S%*%t(svd$V)

0.1428571,0,0.000000,0
1.0000000,4,0.000000,0
3.1415927,-2,1.414214,0
1.4142136,3,0.000000,0


0.1428571,-1.734723e-18,-6.938894e-18,0
1.0000000,4.000000e+00,0.000000e+00,0
3.1415927,-2.000000e+00,1.414214e+00,0
1.4142136,3.000000e+00,-5.551115e-17,0


**Principales hallazos del test 10**

La función arroja resultados correctos encontrando la SVD de A.

**Test 11**

**Objetivo del test 11:** Verificar que la función se comporta correctamente para matrices A **ortogonales**.

**Implementación del test 11:**

In [65]:
#Definimos A matriz ortogonal
theta<-pi/4
A <- matrix(c(cos(theta),sin(theta),-sin(theta),cos(theta)), nrow=2, ncol=2, byrow = T)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [66]:
A
svd$U%*%svd$S%*%t(svd$V)

0.7071068,0.7071068
-0.7071068,0.7071068


0.7071068,0.7071068
-0.7071068,0.7071068


**Principales hallazos del test 11**

La función arroja resultados correctos encontrando la SVD de A.

**Test 12**

**Objetivo del test 12:** Verificar que la función se comporta correctamente para matrices A **pseudoaleatorias**.

**Implementación del test 12:**

Para la creación de matrices pseudoaleatorias, haremos uso del paquete **mlsjunkgen** que dado un conjunto de input seeds: w,x,y,z genera matrices de números pseudo-aleatorios entre 0 y 1.

In [67]:
install.packages("mlsjunkgen",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: mlsjunkgen, /tmp/RtmpiMujwy/downloaded_packages/mlsjunkgen_0.1.1.tar.gz

files: /tmp/RtmpiMujwy/downloaded_packages/mlsjunkgen_0.1.1.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmpiMujwy/downloaded_packages/mlsjunkgen_0.1.1.tar.gz'



In [68]:
library(mlsjunkgen)

In [72]:
w <- 1
x <- 5
y <- 3
z <- 4

n<-10**1
m<-10**2

#Definimos A matriz pseudo-aleatoria con 16 decimales
A<-mlsjunkgenm(nrow = n, ncol = m, w = w, x = x, y = y, z = z, round = 16)

In [73]:
#Calculamos SVD
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

**Comprobamos que estas matrices son una SVD de A:**

In [74]:
A
svd$U%*%svd$S%*%t(svd$V)

0.847093986,0.9713891,0.8645104,0.28206127,0.78764684,0.45964341,0.4361059,0.438944685,0.42980199,0.01005201,⋯,0.74882670,0.8188853,0.989349674,0.49023124,0.97576613,0.02157832,0.76989782,0.65482397,0.7021928,0.71941684
0.746115227,0.2617650,0.6417098,0.70219179,0.36677052,0.52111346,0.5854412,0.056552321,0.05502361,0.84160455,⋯,0.05447558,0.3645880,0.904140326,0.65201960,0.15020663,0.18483044,0.26247519,0.20666534,0.6172786,0.70417696
0.436393491,0.7464926,0.5071930,0.70899268,0.98794839,0.45742096,0.8007255,0.612099773,0.17579378,0.27279581,⋯,0.34716929,0.7755368,0.001245041,0.07234552,0.91508382,0.08790926,0.79592559,0.88401396,0.6684382,0.40465812
0.003548722,0.1802593,0.1615858,0.26811666,0.85106955,0.79418897,0.4221502,0.603443788,0.23405934,0.23172384,⋯,0.52805271,0.9417318,0.632140143,0.54758199,0.39665862,0.02055855,0.16395987,0.83004709,0.2921833,0.12434679
0.262946911,0.5545452,0.3542665,0.03499987,0.07751327,0.95024571,0.6641080,0.775671734,0.70373065,0.47395619,⋯,0.78832661,0.4253929,0.680083204,0.54389721,0.86239717,0.63226288,0.37929488,0.67474536,0.3536653,0.46241680
0.324979464,0.5617152,0.3767844,0.40179265,0.26776372,0.54135361,0.8696630,0.298280690,0.59194178,0.16926407,⋯,0.08068649,0.1915935,0.150201330,0.96017753,0.19802047,0.77022926,0.47367800,0.26243975,0.3274193,0.97928390
0.622638935,0.1296170,0.4159080,0.04488852,0.59169089,0.45936268,0.9651222,0.860343546,0.89873455,0.42812996,⋯,0.75523578,0.4340762,0.115650597,0.82741591,0.05444877,0.58248787,0.42891557,0.48317931,0.1326457,0.13997958
0.118732307,0.5340533,0.6795494,0.96684736,0.28750470,0.07005899,0.9454620,0.538394625,0.47723770,0.66889826,⋯,0.41111114,0.5945681,0.573897145,0.76166239,0.94460606,0.64410071,0.17243377,0.97231242,0.3712654,0.99718068
0.733660081,0.9825798,0.1018399,0.01974215,0.65805039,0.58552912,0.1830504,0.699020421,0.05117424,0.32957337,⋯,0.85311347,0.8587276,0.760295039,0.41981185,0.25307472,0.42000691,0.03837538,0.05813527,0.7954330,0.09108484
0.039080997,0.3026420,0.5166428,0.32624773,0.13347826,0.35099209,0.4242837,0.005065657,0.08016398,0.14428638,⋯,0.66653807,0.2004193,0.422205387,0.69730008,0.00503290,0.73931438,0.42735495,0.20582356,0.7614863,0.01202526


0.847093986,0.9713891,0.8645104,0.28206127,0.78764684,0.45964341,0.4361059,0.438944685,0.42980199,0.01005201,⋯,0.74882670,0.8188853,0.989349674,0.49023124,0.97576613,0.02157832,0.76989782,0.65482397,0.7021928,0.71941684
0.746115227,0.2617650,0.6417098,0.70219179,0.36677052,0.52111346,0.5854412,0.056552321,0.05502361,0.84160455,⋯,0.05447558,0.3645880,0.904140326,0.65201960,0.15020663,0.18483044,0.26247519,0.20666534,0.6172786,0.70417696
0.436393491,0.7464926,0.5071930,0.70899268,0.98794839,0.45742096,0.8007255,0.612099773,0.17579378,0.27279581,⋯,0.34716929,0.7755368,0.001245041,0.07234552,0.91508382,0.08790926,0.79592559,0.88401396,0.6684382,0.40465812
0.003548722,0.1802593,0.1615858,0.26811666,0.85106955,0.79418897,0.4221502,0.603443788,0.23405934,0.23172384,⋯,0.52805271,0.9417318,0.632140143,0.54758199,0.39665862,0.02055855,0.16395987,0.83004709,0.2921833,0.12434679
0.262946911,0.5545452,0.3542665,0.03499987,0.07751327,0.95024571,0.6641080,0.775671734,0.70373065,0.47395619,⋯,0.78832661,0.4253929,0.680083204,0.54389721,0.86239717,0.63226288,0.37929488,0.67474536,0.3536653,0.46241680
0.324979464,0.5617152,0.3767844,0.40179265,0.26776372,0.54135361,0.8696630,0.298280690,0.59194178,0.16926407,⋯,0.08068649,0.1915935,0.150201330,0.96017753,0.19802047,0.77022926,0.47367800,0.26243975,0.3274193,0.97928390
0.622638935,0.1296170,0.4159080,0.04488852,0.59169089,0.45936268,0.9651222,0.860343546,0.89873455,0.42812996,⋯,0.75523578,0.4340762,0.115650597,0.82741591,0.05444877,0.58248787,0.42891557,0.48317931,0.1326457,0.13997958
0.118732307,0.5340533,0.6795494,0.96684736,0.28750470,0.07005899,0.9454620,0.538394625,0.47723770,0.66889826,⋯,0.41111114,0.5945681,0.573897145,0.76166239,0.94460606,0.64410071,0.17243377,0.97231242,0.3712654,0.99718068
0.733660081,0.9825798,0.1018399,0.01974215,0.65805039,0.58552912,0.1830504,0.699020421,0.05117424,0.32957337,⋯,0.85311347,0.8587276,0.760295039,0.41981185,0.25307472,0.42000691,0.03837538,0.05813527,0.7954330,0.09108484
0.039080997,0.3026420,0.5166428,0.32624773,0.13347826,0.35099209,0.4242837,0.005065657,0.08016398,0.14428638,⋯,0.66653807,0.2004193,0.422205387,0.69730008,0.00503290,0.73931438,0.42735495,0.20582356,0.7614863,0.01202526


Haremos uso del paquete tictoc para medir los tiempos

In [76]:
install.packages("tictoc",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: tictoc, /tmp/RtmpiMujwy/downloaded_packages/tictoc_1.0.tar.gz

files: /tmp/RtmpiMujwy/downloaded_packages/tictoc_1.0.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmpiMujwy/downloaded_packages/tictoc_1.0.tar.gz'



In [77]:
library(tictoc)

**Y verificamos que la función encuentra solución para A pseudo-aleatoria de $10^1 \times 10^2$ en 47 segundos**

In [78]:
tic("medición de sleep + svd")
tic("medición de tiempo de svd con tictoc")
tic()
svd<-svd_jacobi_aprox(A,TOL,maxsweep)
toc()
Sys.sleep(1)
toc()

medición de tiempo de svd con tictoc: 46.725 sec elapsed
medición de sleep + svd: 47.732 sec elapsed


**Principales hallazos del test 12**

* La función arroja resultados correctos para matrices pseudoaleatorias.

**4. Resumen detallado de posibles puntos faltantes en implementación**

* Se sugiere corregir un typo en el comentario de la función: #inicialza valores del ciclo por #inicializa valores del ciclo
* Necesita revisarse la función para A matrices renglón pues arrojan el mismo error que en los Test 1 y Test 2 comentados en la parte 1 de esta revisión.
* Para el caso de matrices columna, aunque la función encuentra U y V ortogonales, parece ser que la matriz sigma que devuelve es la matriz identidad en lugar de devolver el valor singular.

**Sugerencias para resolver los puntos anteriores**

Se propone ajustar la función con respecto a cada uno de los puntos mencionados de la siguiente forma:

* Se sugiere corregir un typo en el comentario de la función: #inicialza valores del ciclo por #inicializa valores del ciclo
* Se sugiere imprimir S para detectar desde qué parte se almacena como la matriz identidad en lugar de tener el valor singular que satisface A = U*S*t(V).